In [5]:
import findspark
findspark.init()

from pathlib import Path
import time

import numpy as np
import pyqtgraph as pg
import pyspark
from pyspark import AccumulatorParam

from janelia_core.fileio.ahrens_lab import read_exp
from janelia_core.fileio.exp_reader import read_img_file

In [6]:
ephys_folder = Path(r'E:\\bishop_work\\ahrens_two_color\\20161109_2_1_6dpf_GFAP_GC_Huc_RG_GA_CL_fb_OL_f0_0GAIN_20161109_211950\\ephys')
image_folder = Path(r'E:\\bishop_work\\ahrens_two_color\\20161109_2_1_6dpf_GFAP_GC_Huc_RG_GA_CL_fb_OL_f0_0GAIN_20161109_211950\raw')

In [7]:
exp = read_exp(image_folder, ephys_folder)

Searching for image files...
Found 12500 images.
Reading ephys data.


In [20]:
n_images = 12500 # Number of images to perform svd on

In [9]:
conf = pyspark.SparkConf().setMaster('local[44]').setAll([
    ('spark.executor.memory', '100g'), ('spark.driver.memory','400g'), ('spark.driver.maxResultSize', '300g')])
sc = pyspark.SparkContext(conf=conf)

In [17]:
def read_and_norm_img(f):
    data = read_img_file(f)
    return data/n_images

In [21]:
t0 = time.time()
mn = sc.parallelize(exp.ts_data['imgs']['vls'][0:n_images]).map(read_and_norm_img).reduce(lambda a,b:a+b)
t1 = time.time(); 
print("Elapsed time: " + str(t1 - t0))

Elapsed time: 2495.691112279892


In [22]:
%gui qt
pg.image(mn)

In [21]:
sc.stop()